# **Mozart**: The ML Models Gradio Demo:

Using Gradio 3.0, this demo is designed to showcase the backend of the Platform.

-Bharath Raj

##Installing Dependencies:

Dependencies for Colab:

In [1]:
#@markdown Enable saving outputs to Google Drive to save your creations at AI/models
save_outputs_to_google_drive = False #@param {type:"boolean"}
#@markdown Enable saving models to Google Drive to avoid downloading the 6GB model every Colab instance
save_models_to_google_drive = True #@param {type:"boolean"}

if save_outputs_to_google_drive or save_models_to_google_drive:
    from google.colab import drive
    try:
      drive.mount('/content/gdrive')
    except:
      save_outputs_to_google_drive = False
      save_models_to_google_drive = False

model_path = "/content/gdrive/MyDrive/AI/models" if save_models_to_google_drive else "/models/ldm/text2img-large"
outputs_path = "/content/gdrive/MyDrive/AI/Latent_Diffusion" if save_outputs_to_google_drive else "/content/outputs"
!mkdir -p $model_path
!mkdir -p $outputs_path
print(f"Model will be stored at {model_path}")
print(f"Outputs will be saved to {outputs_path}")


Mounted at /content/gdrive
Model will be stored at /content/gdrive/MyDrive/AI/models
Outputs will be saved to /content/outputs


In [2]:
%%capture
!git clone https://github.com/crowsonkb/latent-diffusion.git
!git clone https://github.com/CompVis/taming-transformers
!pip install -e ./taming-transformers
!pip install omegaconf>=2.0.0 pytorch-lightning>=1.0.8 torch-fidelity einops
!pip install transformers
!pip install open_clip_torch
!pip install autokeras
!pip install tensorflow
!pip install gradio
import sys
sys.path.append(".")
sys.path.append('./taming-transformers')
from taming.models import vqgan 

%cd /content/latent-diffusion

import os
if os.path.isfile(f"{model_path}/latent_diffusion_txt2img_f8_large.ckpt"):
    print("Using saved model from Google Drive")
else:    
    !wget -O $model_path/latent_diffusion_txt2img_f8_large.ckpt https://ommer-lab.com/files/latent-diffusion/nitro/txt2img-f8-large/model.ckpt

## Importing Modules for Style Transfer

In [3]:
import os
import tensorflow as tf
import numpy as np
import PIL.Image
import time
import functools
import tensorflow_hub as hub


##Importing Modules for Super Resolution

In [4]:
import urllib.request
from PIL import Image
import requests
import shutil

##Importing Modules for Latent Diffusion

In [5]:
import torch
from omegaconf import OmegaConf
from ldm.util import instantiate_from_config
import argparse, os, sys, glob
import numpy as np
from PIL import Image
from tqdm.auto import tqdm, trange
tqdm_auto_model = __import__("tqdm.auto", fromlist=[None]) 
sys.modules['tqdm'] = tqdm_auto_model
from einops import rearrange
from torchvision.utils import make_grid
import transformers
import gc
from ldm.util import instantiate_from_config
from ldm.models.diffusion.ddim import DDIMSampler
from ldm.models.diffusion.plms import PLMSSampler
from open_clip import tokenizer
import open_clip
import tensorflow as tf


##Function Definitions:

#####Style Transfer:

In [6]:
hub_model = hub.load("https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2")
def style_transfer(content_image, style_image):
    content_image = tf.convert_to_tensor(content_image, np.float32)[tf.newaxis, ...] / 255.
    style_image = tf.convert_to_tensor(style_image, np.float32)[tf.newaxis, ...] / 255.
    output = hub_model(content_image, style_image)
    stylized_image = output[0]
    return Image.fromarray(np.uint8(stylized_image[0] * 255))


#####Latent Diffusion LAION400:

In [7]:
def load_safety_model(clip_model):
    """load the safety model"""
    import autokeras as ak  # pylint: disable=import-outside-toplevel
    from tensorflow.keras.models import load_model  # pylint: disable=import-outside-toplevel
    from os.path import expanduser  # pylint: disable=import-outside-toplevel

    home = expanduser("~")

    cache_folder = home + "/.cache/clip_retrieval/" + clip_model.replace("/", "_")
    if clip_model == "ViT-L/14":
        model_dir = cache_folder + "/clip_autokeras_binary_nsfw"
        dim = 768
    elif clip_model == "ViT-B/32":
        model_dir = cache_folder + "/clip_autokeras_nsfw_b32"
        dim = 512
    else:
        raise ValueError("Unknown clip model")
    if not os.path.exists(model_dir):
        os.makedirs(cache_folder, exist_ok=True)

        from urllib.request import urlretrieve  # pylint: disable=import-outside-toplevel

        path_to_zip_file = cache_folder + "/clip_autokeras_binary_nsfw.zip"
        if clip_model == "ViT-L/14":
            url_model = "https://raw.githubusercontent.com/LAION-AI/CLIP-based-NSFW-Detector/main/clip_autokeras_binary_nsfw.zip"
        elif clip_model == "ViT-B/32":
            url_model = (
                "https://raw.githubusercontent.com/LAION-AI/CLIP-based-NSFW-Detector/main/clip_autokeras_nsfw_b32.zip"
            )
        else:
            raise ValueError("Unknown model {}".format(clip_model))
        urlretrieve(url_model, path_to_zip_file)
        import zipfile  # pylint: disable=import-outside-toplevel

        with zipfile.ZipFile(path_to_zip_file, "r") as zip_ref:
            zip_ref.extractall(cache_folder)

    loaded_model = load_model(model_dir, custom_objects=ak.CUSTOM_OBJECTS)
    loaded_model.predict(np.random.rand(10 ** 3, dim).astype("float32"), batch_size=10 ** 3)

    return loaded_model

def is_unsafe(safety_model, embeddings, threshold=0.5):
    """find unsafe embeddings"""
    nsfw_values = safety_model.predict(embeddings, batch_size=embeddings.shape[0])
    x = np.array([e[0] for e in nsfw_values])
    #print(x)
    return True if x > threshold else False
#NSFW CLIP Filter
safety_model = load_safety_model("ViT-B/32")
clip_model, _, preprocess = open_clip.create_model_and_transforms('ViT-B-32', pretrained='openai')

def load_model_from_config(config, ckpt, verbose=False):
    print(f"Loading model from {ckpt}")
    pl_sd = torch.load(ckpt, map_location="cuda:0")
    sd = pl_sd["state_dict"]
    model = instantiate_from_config(config.model)
    m, u = model.load_state_dict(sd, strict=False)
    if len(m) > 0 and verbose:
        print("missing keys:")
        print(m)
    if len(u) > 0 and verbose:
        print("unexpected keys:")
        print(u)

    model = model.half().cuda()
    model.eval()
    return model

config = OmegaConf.load("configs/latent-diffusion/txt2img-1p4B-eval.yaml") 
model = load_model_from_config(config, f"{model_path}/latent_diffusion_txt2img_f8_large.ckpt")

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model = model.to(device)
def run(opt):
    torch.cuda.empty_cache()
    gc.collect()
    if opt.plms:
        opt.ddim_eta = 0
        sampler = PLMSSampler(model)
    else:
        sampler = DDIMSampler(model)
    
    os.makedirs(opt.outdir, exist_ok=True)
    outpath = opt.outdir

    prompt = opt.prompt


    sample_path = os.path.join(outpath, "samples")
    os.makedirs(sample_path, exist_ok=True)
    base_count = len(os.listdir(sample_path))

    all_samples=list()
    with torch.no_grad():
        with torch.cuda.amp.autocast():
            with model.ema_scope():
                uc = None
                if opt.scale > 0:
                    uc = model.get_learned_conditioning(opt.n_samples * [""])
                for n in trange(opt.n_iter, desc="Sampling"):
                    c = model.get_learned_conditioning(opt.n_samples * [prompt])
                    shape = [4, opt.H//8, opt.W//8]
                    samples_ddim, _ = sampler.sample(S=opt.ddim_steps,
                                                    conditioning=c,
                                                    batch_size=opt.n_samples,
                                                    shape=shape,
                                                    verbose=False,
                                                    unconditional_guidance_scale=opt.scale,
                                                    unconditional_conditioning=uc,
                                                    eta=opt.ddim_eta)

                    x_samples_ddim = model.decode_first_stage(samples_ddim)
                    x_samples_ddim = torch.clamp((x_samples_ddim+1.0)/2.0, min=0.0, max=1.0)

                    for x_sample in x_samples_ddim:
                        x_sample = 255. * rearrange(x_sample.cpu().numpy(), 'c h w -> h w c')
                        image_vector = Image.fromarray(x_sample.astype(np.uint8))
                        image = preprocess(image_vector).unsqueeze(0)
                        with torch.no_grad():
                          image_features = clip_model.encode_image(image)
                        image_features /= image_features.norm(dim=-1, keepdim=True)
                        query = image_features.cpu().detach().numpy().astype("float32")
                        unsafe = is_unsafe(safety_model,query,opt.nsfw_threshold)
                        if(not unsafe):
                          image_vector.save(os.path.join(sample_path, f"{base_count:04}.png"))
                        else:
                          raise Exception('Potential NSFW content was detected on your outputs. Try again with different prompts. If you feel your prompt was not supposed to give NSFW outputs, this may be due to a bias in the model')
                        base_count += 1
                    all_samples.append(x_samples_ddim)


    grid = torch.stack(all_samples, 0)
    grid = rearrange(grid, 'n b c h w -> (n b) c h w')
    grid = make_grid(grid, nrow=opt.n_samples)

    grid = 255. * rearrange(grid, 'c h w -> h w c').cpu().numpy()
    
    Image.fromarray(grid.astype(np.uint8)).save(os.path.join(outpath, f'{prompt.replace(" ", "-")}.png'))
    display(Image.fromarray(grid.astype(np.uint8)))
    return Image.fromarray(grid.astype(np.uint8))


  0%|                                               | 0.00/354M [00:00<?, ?iB/s]

Loading model from /content/gdrive/MyDrive/AI/models/latent_diffusion_txt2img_f8_large.ckpt
LatentDiffusion: Running in eps-prediction mode
DiffusionWrapper has 872.30 M params.
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 4, 32, 32) = 4096 dimensions.
making attention of type 'vanilla' with 512 in_channels


In [34]:
def generate(prompt, quality, resolution):
    Prompt = "an abstract painting of " +prompt
    Steps = 25*{"normal": 2, "better": 3}[quality]
    ETA = 0.0 
    Iterations = 1
    Width=256*{"normal": 1, "better": 2}[quality]
    Height=256*{"normal": 1, "better": 2}[quality]
    Samples_in_parallel=1 
    Diversity_scale=5.0
    PLMS_sampling=True
    args = argparse.Namespace(
      prompt = Prompt, 
      outdir=f'{outputs_path}',
      ddim_steps = Steps,
      ddim_eta = ETA,
      n_iter = Iterations,
      W=Width,
      H=Height,
      n_samples=Samples_in_parallel,
      scale=Diversity_scale,
      plms=PLMS_sampling,
      nsfw_threshold=0.5
    )
    return run(args)

#####Super Resolution:

In [10]:
def return_resolution(path):
  img = Image.open(path)
  w, h = img.size
  return (str(w) + "x" + str(h))


def enhance(path):
  Image.fromarray(path.astype('uint8'), 'RGB').save('temp.jpg', 'jpeg')
  before = return_resolution("temp.jpg")
  r = requests.post(
    "https://api.deepai.org/api/torch-srgan",
    files={
        'image': open('temp.jpg', 'rb')
    },
    headers={'api-key': 'quickstart-QUdJIGlzIGNvbWluZy4uLi4K'}
  )
  urllib.request.urlretrieve(r.json()["output_url"],"temp.jpg")
  img = Image.open("temp.jpg")
  after = return_resolution("temp.jpg")
  return img, "Previous Resolution: "+before+"\tSuper Resolution: "+after
  

##### Gallery:

In [11]:
def imgur_retreive(key):
  urllib.request.urlretrieve("https://i.imgur.com/"+key+".png",key+".jpg")
  return key+".jpg"

  
from glob import glob
imgur_keys = [ "srWawat", "tFXbZfI",  "6q6k737", "6mEeFQz", "zei8pqq"]
list_of_images = [imgur_retreive(i) for i in imgur_keys]

def infer():
    return list_of_images

#Gradio Interface:



#####Markdown Documentation:

In [14]:
documentation_md = """
 
<br> 
</br>

Since prehistoric times, human beings have closely linked any and all forms of their work with some sort of artistic interpretation. Art, in all of its forms, plays a major role in how humans see and interact with each other and their environment. The earliest known examples of art are [dated to 30000 BP](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2815939/), during the later stone age. This implies how art, in all its forms, allows us to express ourselves and take in perspectives that cannot be expressed via words. Ever since its inception, art has crossed any and all limitations, from physical to intangible forms, i.e., people of all cultures, races, and ethnicities have some form of message or interpretation of any kind of artwork in their respective culture. Art has not only influenced aesthetes and artists to make more of it but has also helped record major historic events and narratives of the past, that would otherwise be long lost to trivial human wars and natural calamities.

  
  
  

Over the years, the enduring appeal of art has changed from just admiration to articulation, intellectual contemplation, and depiction of the state of human society. However, the thing that abstract art stands for, which is triggering an emotional response from its viewers and letting them ponder about its meaning, is still very much intact. Abstract art doesn't necessarily depict anything that the real world has. It can simply be an expression of language or emotion using colors, patterns forms and styles. The best part about art is that all kinds of people experience it differently. One might simply admire the color schemes of a painting, while the other may look at the strokes and patterns it leads to. It starts out as a certain thought in the hands of the artist, and ends up as a completely different, unique thought in the eyes of the beholder.

  

<center>

  

<img src="https://gohighbrow.com/wp-content/uploads/2015/05/10-No.-5.jpg" alt="Jackson Pollock's No.5, 1948" style="zoom:50%;" />

  

​ [_No.5 by Jackson Pollock_](https://www.jackson-pollock.org/number-5.jsp)

</center>

  
  
  
  

# Recent AI Breakthroughs

  

Although the role of AI in art creation and the relationship between artists and AI is still evolving, some very significant leaps have been made during the past decade, and remarkable feats like [computers beating humans at image recognition](https://image-net.org/challenges/LSVRC/2015/ilsvrc2015_perclass_results.tar.gz), at complex games with a mathematically large amount of patterns and outcomes like [chess and go](https://arxiv.org/pdf/1712.01815.pdf), [AI models generating text](https://arxiv.org/pdf/2005.14165.pdf) and [artistic images](https://arxiv.org/ftp/arxiv/papers/2205/2205.02439.pdf) have already been achieved. It's extremely intriguing how well AI can mimic and surpass human capabilities in the aforementioned activities.

  
  
  
  

## Deep Learning

  

Deep learning is a subset of machine learning inspired by the brain's network of neurons. And while neural networks have been around for decades, one of the most well known advancements for deep learning happened in 2012, called the [ImageNet Large Scale Visual Recognition Challenge (ILSVRC)](https://arxiv.org/pdf/1409.0575.pdf). It was a pivotal moment for the use of deep neural networks for image recognition.

  

Another famous model is [Google's BERT](https://arxiv.org/pdf/1810.04805.pdf), a language model based on [transformers](https://arxiv.org/pdf/1706.03762.pdf) to help the search engine get better context around its users' searches. It was released in 2018 with 12 layers, 12 attention heads and 110 million parameters. [Language models based on RNN](https://www.researchgate.net/publication/221489926_Recurrent_neural_network_based_language_model#read) faced issues like parallelizing and retaining contextual connections. Since context is key in [NLP](https://watermark.silverchair.com/18-5-544.pdf?token=AQECAHi208BE49Ooan9kkhW_Ercy7Dm3ZL_9Cf3qfKAc485ysgAAAskwggLFBgkqhkiG9w0BBwagggK2MIICsgIBADCCAqsGCSqGSIb3DQEHATAeBglghkgBZQMEAS4wEQQMsJn4dn7kYS2Aa0UQAgEQgIICfOKaz4s0K6dqYDOfg5tKrCN6LEMFykUpzqVQI9713w5Az_zchAA_d6SaKh3cy6TL4SXckDQy4TftOb2e5Ke7lQ3jk2GuIzNH5X5kJNhXJ5SDshTqZG-1B93HB06gNpil8i9UlKiai5-Av8uLVdoaGmo0S-zaby1D7wF3EfubvoQqNcTviPphiV1oBYe8P9HQPt_C2WrqDlAekP6O8Q6LoXCIp28q4fDalRhEmAnWJWa8dlRsPyuent-JFBl5PBJwGfmd4bTcRtphxixw16WIRktejkScr4JYE-wMmOZiT3GrSp3o_olgdwfM8zoILgBoS60rYwiyGaMl9_XzlVyg6RHVa3xSHRHoeDqJHnB0Aq0aHzsSQnHufkA7jmmSgwy9xWlR7QwJDp2POW_51pyhMyKPetxcRVQSLVUPX3e2rgJJU5tLnc21Yz9zDdntQ8zxPdR4tEskUUwkbeONfj9n2yirvpD2arhws2n6nmo90CT04ppytxMYagIuV3aSGcao2boKMFr4_iD30qv1yjZO9qT4UTOJX51ZsYjyeY0VcVzT76GnvoMm79nCTGAr8Pk0hRG7yMZXhjjGViXek4jLwZSWXLgDT0Rrgg4a_ZtY6M6amJhEVjHyR5Qu1cDjdVBm6PvsuUZvQzU03UiJgg86bplT2vN6GyJkDP5C4PFXUpS3CGy_GzPxtrIKOBZHvSZNQVym2imOB4bivfBfm66Pedq_oevLjW3Lw06aMIx7ucXYYBI1V8EDD2mAf_tTzp-qkcfht74EbBc91cxKP2tV8YJ6KwcZTIs28WFco3k8vu58iPyn5_i7aKB8Cr_GLAC2fITBIj6RQMYEDYT2ng), BERT easily bypassed these problems, and the results were far better than the state-of-the-art RNN models. But this was just the beginning of transformer based neural networks, as Open AI's GPT-3 was shockingly good and even better than BERT.

  
  
  

## GPT-3: AI gets language

  

On 11 June 2020, [OpenAI](https://openai.com/about/) released the biggest language model ever known, [GPT-3](https://openai.com/api/) along with its [API documentation](https://beta.openai.com/docs/). Trained on 175 billion parameters, which is 116 times more than it's predecessor GPT-2, the language prediction model has the capability to serve a wide range of purposes such as creating blog posts, advertisements, even poetry that mimics the style of famous poets like Shakespeare, Edgar Allan Poe, so realistic that it can easily be looked at as something written by an actual human being. It can even generate text summarizations and code. GPT-3 is trained to perform such tasks with [small amounts of input text while maintaining context and producing large amounts of output](https://cdn.openai.com/papers/Training_language_models_to_follow_instructions_with_human_feedback.pdf).

  

Around September 2020, The Guardian had used GPT-3 to write [an article about AI being harmless to human beings](https://www.theguardian.com/commentisfree/2020/sep/08/robot-wrote-this-article-gpt-3). The article really shows its proficiency, and is a spectacle to witness in the world of language generation.

  
  
  

## DALL-E: AI gets artsy

  

[DALL-E](https://openai.com/blog/dall-e/) is a multimodal implementation of GPT-3 with 12 billion parameters, which uses [zero-short learning](https://arxiv.org/pdf/2102.12092.pdf) to generate images and art based on text prompts of descriptions and desired styles. Trained on text-image pairs from the internet, DALL-E creates multiple responses to any text input, and [CLIP](https://arxiv.org/pdf/2103.00020.pdf), an image recognition system, understands and ranks these images to associate the most appropriate image to the caption (text).

  
  
  
  
  
  
  
  
  
  
  

# Generative Art

  

As inferred from [Prof. Philip Galanter's paper](http://www.philipgalanter.com/downloads/ga2003_paper.pdf), the term 'Generative Art' refers to the kind of art where the artist uses a system, like a set of natural language rules, a computer program, a machine, or other procedural invention, set into motion with some degree of autonomy contributing or resulting in a finished work of art. An [autonomous system](http://www.generativeart.com/on/cic/papersga2007/07.pdf) in the context of generative art is a system that is non-human and can independently determine features of an artwork that would otherwise require decisions made directly by the artist.

  

One of the first known examples of generative art is a musical game called **Musikalisches Würfelspiel** in which dice were rolled to randomly select already-composed fragments of music, which were strung together to form a finished piece. This game was played in Berlin in 1792, and has been attributed to the famous composer Wolfgang Amadeus Mozart, hence, this being the inspiration behind the model being named 'Mozart'.

  

Another example of generative art is the set of Italian Medieval town designs created by an architect named Celestino Soddu in 1987. He created a set of conditions where a random computer process could be set in motion to create a model of a town. The conditions were such that the final result would always be a town identifiable in the Italian Medieval style. Despite there being enough constraints on the models to keep them in this style, an essentially infinite number of models could be created.

  

As shown in the above example, the beauty of generative art is that the possibilities for creation are limitless. A process with certain conditions created by the user are set into motion and the element of chance creates the most intruiging piece of digital art.

  

During the 1960s, when generative art became quite popular, famous digital artist [Harold Cohen](https://computerhistory.org/blog/harold-cohen-and-aaron-a-40-year-collaboration/) became interested in work by computer scientists at the University of San Diego. Programmers created a system on punched cards, fed those cards into a machine which would then return results via a set of newly punched cards, or prints. He applied this technology to computer-controlled drawing machines, which he called "turtles". Cohen programmed the “turtles” to follow a set of processes which would lead to a piece of art being formed on canvas. His system, [AARON](http://www.haroldcohen.com/aaron/publications/how2make.pdf), is one of the longest-running, continually maintained AI systems in history.

  

<img src="http://images.computerhistory.org/blog-media/harold_cohen_aaron_IMG_6638.jpg" alt="Detail from an untitled AARON drawing, ca. 1980." style="zoom:50%;" />

  

​ [_Detail from an untitled AARON drawing, ca. 1980._](https://images.computerhistory.org/blog-media/harold_cohen_aaron_IMG_6638.jpg)

  

However, Cohen has further explained in his paper - [The further exploits of AARON, Painter](https://web.archive.org/web/20060107184824/http://crca.ucsd.edu/~hcohen/cohenpdf/furtherexploits.pdf) regarding the artistic legitimacy and functions of what his system is exactly doing, and whether it's justified to call it 'creative'.

  

Another prominent artist named Vera Molnar, during the same time as Cohen, had been experimenting with creating images using "machine imaginaire", as she called it. She created a set of rules to paint a series of geometric images, allowing her to explore endless shapes and lines.

  

<img src="https://images.squarespace-cdn.com/content/v1/59413d96e6f2e1c6837c7ecd/1534897464622-0LL0T4WAII94R4FIV49B/Interruptions+-+Vera+Moln%C3%A1r%2C+1968%2F69?format=750w" alt="img" style="zoom:50%;" />

  

​ [_Interruptions - Vera Molnar_](https://images.squarespace-cdn.com/content/v1/59413d96e6f2e1c6837c7ecd/1534897464622-0LL0T4WAII94R4FIV49B/Interruptions+-+Vera+Molnár%2C+1968%2F69?format=750w)

  
  
  

"""

# User Interface

In [35]:
import gradio as gr

ga_description = """
<center>
<img src=https://i.imgur.com/lolHhBs.png width=700px>
</center>
"""
st_description = """
<center>
<img src=https://i.imgur.com/goqyoWH.png width=700px>
</center>
"""
sr_description = """
<center>
<img src=https://i.imgur.com/RhjYIZh.png width=700px>
</center>
"""
gl_description = """
<center>
<img src=https://i.imgur.com/ncvkiph.png width=700px>
</center>
"""
prompt = gr.Textbox(label="Text Prompt")
quality = gr.Radio(choices=['normal', 'better'], label="Quality")
resolution = gr.Radio(choices=['normal', 'better'], label="Resolution")
starry_night = [imgur_retreive("dqpGGLm"),imgur_retreive("MHTDaZ3")]
content = gr.Image(label="Content Image")
style = gr.Image(shape=(256, 256), label="Style Image")
original = gr.Image(shape=None, image_mode="RGB", invert_colors=False, source="upload", tool="editor", type="numpy", label="content")
text_output = gr.Textbox(show_label=False)
gallery=gr.Gallery(label="Mozart Gallery").style(grid=[5], height="auto")

with gr.Blocks() as iFace:
  with gr.Tabs():
    with gr.TabItem("Generate Art"):
      gr.Interface(generate,
                     inputs = [prompt, quality, resolution],
                     outputs = "image",
                     allow_flagging = "never",
                     description = ga_description,
                     article = "Save the Art to Gallery if you like it!",
                     live=False
                     )
      gr.Button(value="Save to gallery")
      
    with gr.TabItem("Style Transfer"):
      editor = gr.Interface(style_transfer, 
                      inputs=[content, style], 
                      description = st_description,
                      examples = [starry_night],
                      article = "Style transfer is a computer vision technique"
                      +" that allows us to recompose the content of an image in the style of another",
                      outputs="image",
                      allow_flagging = "never",
                      live=False)
    with gr.TabItem("Super Resolution"):
      gr.Interface(enhance, 
                      inputs=[original], 
                      description = sr_description,
                      article = "2x your image resolution",
                      outputs=["image", text_output],
                      allow_flagging = "never",
                      live=False)
    with gr.TabItem("Gallery"):
      gr.Interface(infer, 
                      inputs=None, 
                      live=True, 
                      allow_flagging="never", 
                      description = gl_description,
                      article = "(Some of the art generated by Mozart",
                      outputs=gallery)
    gr.Markdown(documentation_md)

iFace.launch(debug=True)



Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://40567.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces: https://huggingface.co/spaces


Keyboard interruption in main thread... closing server.


(<gradio.routes.App at 0x7fa1200a7f50>,
 'http://127.0.0.1:7860/',
 'https://40567.gradio.app')

Clean Cache:

In [ ]:
torch.cuda.empty_cache()